In [5]:
import pandas as pd
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt



In [6]:
path = 'E:/Dataset/新闻推荐比赛数据/'
train = path + 'articles.csv'
train_click = path + 'train_click_log.csv'

In [7]:
# 分析用户本身的基本属性，文章用户和文章交互的一些分布，召回策略，特征工程
#####train
trn_click = pd.read_csv(path+'train_click_log.csv')
#trn_click = pd.read_csv(path+'train_click_log.csv', names=['user_id','item_id','click_time','click_environment','click_deviceGroup','click_os','click_country','click_region','click_referrer_type'])
item_df = pd.read_csv(path+'articles.csv')
item_df = item_df.rename(columns={'article_id': 'click_article_id'})  #重命名，方便后续match
# item_emb_df = pd.read_csv(path+'articles_emb.csv')